In [ ]:
import pandas as pd
import numpy as np
import scipy as sci
import arch
import pyvinecopulib as pv
import matplotlib.pyplot as plt
import plotly.express as px
from SourceCodes.invariance_analysis import quick_invariance_analysis


columns_mappings = {'ES1 Index': 'SP500',
                    'NQ1 Index': 'Nasdaq100',
                    'VG1 Index': 'Euro Stox50',
                    'BZ1 Index': 'Ibovespa',
                    'TY1 Comdty':'10-Year Treasury',
                    'RX1 Comdty': 'Euro Bund',
                    'EC1 Curncy': 'EUR/USD',
                    'BP1 Curncy': 'GBP/USD',
                    'UC1 Curncy': 'USD/BRL',
                    'CL1 Comdty': 'WTI',
                    'CO1 Comdty': 'Brent'}

selected_securities = ['SP500', 'Nasdaq100', 'Ibovespa', '10-Year Treasury','USD/BRL']


In [156]:
# Reading data
df = pd.read_csv("Data/data.csv", index_col="Dates", date_parser = pd.to_datetime).rename(columns=columns_mappings)[selected_securities]
df.sort_index(inplace=True)
returns_scale_factor = 100
df_log_ret = np.log(df).diff() * returns_scale_factor
n = df.shape[0]
n_os = 252
n_scenarios = 1000
seeds = [int(x) for x in np.ones(len(selected_securities)).tolist()]

In [ ]:
# Quick Invariance Analysis
for item in df_log_ret.columns:
    fig, axs = quick_invariance_analysis(df_log_ret[item].iloc[:-n_os], n_chunks=3, nbins=30)
    fig.set_size_inches(20, 20)

In [ ]:
# Fitting a GARCH(1,1) Model and
model_list = []

for k, item in enumerate(df_log_ret):
    mdl = list()
    mdl.append(arch.arch_model(df_log_ret.iloc[1:-n_os, k] , mean="Zero", vol="GARCH", p=1, q=1, dist="t"))
    mdl.append(mdl[0].fit())
    model_list.append(mdl)

In [ ]:
fig = model_list[0][1].plot(scale=252)
fig.set_size_inches(10, 10)

In [ ]:
# Creating the matrix of residuals and scatter plotting
for k, item in enumerate(model_list):
    if k == 0:
        df_log_resid = pd.DataFrame(item[1].std_resid)
        df_log_resid.rename(columns={df_log_resid.columns[-1]:item[0].y.name}, inplace=True)
    else:
        df_log_resid = pd.concat([df_log_resid, pd.DataFrame(item[1].std_resid)], axis=1)
        df_log_resid.rename(columns={df_log_resid.columns[-1]:item[0].y.name}, inplace=True)

In [ ]:
# Quick Invariance Analysis - Of residuals
for item in df_log_resid.columns:
    fig, axs = quick_invariance_analysis(df_log_resid[item], n_chunks=3, nbins=30)
    fig.set_size_inches(20, 20)

In [ ]:
# Calculating the pseudo-observations - Estimated Probability Integral Transform
for k, item in enumerate(model_list):
    if k == 0:
        df_log_resid_pseudo_observations = pd.DataFrame(sci.stats.t.cdf(df_log_resid.iloc[:, k], df=model_list[k][1].params.nu))
    else:
        df_log_resid_pseudo_observations = pd.concat([df_log_resid_pseudo_observations, pd.Series(sci.stats.t.cdf(df_log_resid.iloc[:, k], df= np.floor(model_list[k][1].params.nu)))], axis=1, ignore_index=True)

df_log_resid_pseudo_observations.columns = df_log_resid.columns
df_log_resid_pseudo_observations.index=df_log_resid.index

In [ ]:
df_log_resid_pseudo_observations.tail(10)

In [ ]:
# Empirical Copula Plotting
fig1 =  px.scatter_matrix(data_frame=df_log_resid_pseudo_observations,
                          dimensions=df_log_resid_pseudo_observations.columns,
                          height=2000, width=2000)
fig1.update_traces(marker=dict(size=4, line=dict(width=1)), opacity=0.6, showlegend=False)
#fig1.update_traces(diagonal_visible=False)
fig1.update_layout(plot_bgcolor = "#ffebe3", colorway = ["#ff774a"], title = "Empirical Copula")
fig1.update_layout({"yaxis"+str(i+1): dict(range = [0, 1]) for i in range(1, len(df.columns))})
fig1.update_layout({"xaxis"+str(i+1): dict(range = [0, 1]) for i in range(1, len(df.columns))})
fig1.update_xaxes(visible=True, showgrid=True)
fig1.update_yaxes(visible=True, showgrid=True)
fig1.write_html("HTML/file.html")
#fig1.show()

In [ ]:
# Meta Distribution - Distribution which is constructed by an arbitrary copula and arbitrary marginal distributions

In [158]:
# Creating Vine - Structure
copVine = pv.Vinecop(d = df_log_resid_pseudo_observations.columns.shape[0])

# Selecting Most Appropriate Model Given pseudo-observations
copVine.select(data=df_log_resid_pseudo_observations)

In [161]:
# Simulating U - Given Vine - Empirical X Simulated Copulas
n_sim = 10000
u_sim = pd.DataFrame(copVine.simulate(n_sim, seeds=seeds), columns=df_log_resid_pseudo_observations.columns)

In [ ]:
# Plotting simulation results given Vine
fig2 =  px.scatter_matrix(data_frame=u_sim,
                          dimensions=u_sim.columns,
                          height=2000, width=2000)
fig2.update_traces(marker=dict(size=4, line=dict(width=1)), opacity=0.6, showlegend=False)
#fig1.update_traces(diagonal_visible=False)
fig2.update_layout(plot_bgcolor = "#ffebe3", colorway = ["#ff774a"], title = "Simulated Data - Given Vine Structure")
fig2.update_layout({"yaxis"+str(i+1): dict(range = [0, 1]) for i in range(1, len(df.columns))})
fig2.update_layout({"xaxis"+str(i+1): dict(range = [0, 1]) for i in range(1, len(df.columns))})
fig2.update_xaxes(visible=True, showgrid=True)
fig2.update_yaxes(visible=True, showgrid=True)
fig2.write_html("HTML/simulated.html")

In [166]:
# Scenario Simulation
scenarios = list()

for k in range(1):
    scenario = dict()
    scenario['u_sim'] = pd.DataFrame(copVine.simulate(n_os), columns=df_log_resid_pseudo_observations.columns)

    scenarios.append(scenario)














In [167]:
scenario['u_sim']

,SP500,Nasdaq100,Ibovespa,10-Year Treasury,USD/BRL
0,0.629553,0.604450,0.205227,0.967480,0.593633
1,0.028100,0.041084,0.011320,0.880569,0.989292
2,0.500215,0.246142,0.667144,0.847189,0.394212
3,0.843561,0.715346,0.423134,0.493626,0.595399
4,0.096327,0.037325,0.360298,0.791365,0.263864
...,...,...,...,...,...
247,0.207450,0.234933,0.048435,0.168363,0.687640
248,0.599112,0.550973,0.549556,0.392095,0.600666
249,0.993203,0.995129,0.942455,0.067676,0.111330
250,0.627263,0.812902,0.927369,0.811062,0.226296


In [175]:
model = model_list[0]

In [178]:
arch.univariate.ARCH(None , mean="Zero", vol="GARCH", p=1, q=1, dist="t")

TypeError: __init__() got an unexpected keyword argument 'mean'

In [177]:
from arch.univariate import  ARCH

ImportError: cannot import name 'ConstantMean' from 'arch' (C:\Users\yuriv\.conda\envs\py37\lib\site-packages\arch\__init__.py)

In [171]:
df_log_ret.iloc[1:-n_os, k].tail(10)

Dates
2022-02-09    1.435639
2022-02-10   -1.768620
2022-02-11   -1.976009
2022-02-14   -0.352017
2022-02-15    1.591673
2022-02-16    0.122992
2022-02-17   -2.159598
2022-02-18   -0.711168
2022-02-21    0.000000
2022-02-22   -1.006553
Name: SP500, dtype: float64

In [ ]:
def

https://www.oreilly.com/library/view/bayesian-statistics-an/9781118359778/OEBPS/c1-sec1-0007.htm

In [ ]:
# Invariance Analysis
fig1 =  px.scatter_matrix(data_frame=df_log_ret.iloc[1:, :],
                          dimensions=df_log_ret.columns[1:],
                          height=4000, width=4000,
                          symbol="sample", color="sample")
fig1.update_traces(marker=dict(size=4, line=dict(width=1)), opacity=0.6, showlegend=False)
#fig1.update_traces(diagonal_visible=False)
fig1.update_layout(plot_bgcolor = "#ffebe3", colorway = ["#ff774a"], title = "Scatter plots")
fig1.update_layout({"yaxis"+str(i+1): dict(range = [-0.15, 0.15]) for i in range(1, len(df.columns))})
fig1.update_layout({"xaxis"+str(i+1): dict(range = [-0.15, 0.15]) for i in range(1, len(df.columns))})
fig1.update_xaxes(visible=True, showgrid=True)
fig1.update_yaxes(visible=True, showgrid=True)
fig1.write_html("HTML/file.html")

fig1.show()